<a href="https://colab.research.google.com/github/rams1987/Essay_Scoring/blob/main/essay_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: install kaggle

!pip install kaggle


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ramasamyseenivasagan","key":"f41bb0714d3baef12d51ba89162604ef"}'}

In [3]:
# prompt: create a kaggle folder

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle competitions download -c learning-agency-lab-automated-essay-scoring-2

100% 11.9M/11.9M [00:01<00:00, 17.9MB/s]
100% 11.9M/11.9M [00:01<00:00, 10.5MB/s]


In [5]:
# prompt: unzip the downloaded file

!unzip learning-agency-lab-automated-essay-scoring-2.zip


Archive:  learning-agency-lab-automated-essay-scoring-2.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [6]:
import os
import numpy as np
import pandas as pd
import spacy

In [7]:
essay_df = pd.read_csv('train.csv')

essay_df.head()

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [9]:
# prompt: get the sentence count, word count, average sentence length, vocabulary size or unique words in the essay and sentiment of the essagy

# Sentence count
essay_df['sentence_count'] = essay_df['full_text'].apply(lambda x: len(x.split('.')))

# Word count
essay_df['word_count'] = essay_df['full_text'].apply(lambda x: len(x.split()))

# Average sentence length
essay_df['average_sentence_length'] = essay_df['word_count'] / essay_df['sentence_count']

# Vocabulary size or unique words
essay_df['unique_words'] = essay_df['full_text'].apply(lambda x: len(set(x.split())))

# Print the results
essay_df.head()


,essay_id,full_text,score,text_length,word_count,avg_word_length,sentence_count,average_sentence_length,unique_words
0,000d118,Many people have car where they live. The thin...,3,2677,498,5.375502,14,35.571429,253
1,000fe60,I am a scientist at NASA that is discussing th...,3,1669,332,5.027108,20,16.600000,177
2,001ab80,People always wish they had the same technolog...,4,3077,550,5.594545,25,22.000000,253
3,001bdc0,"We all heard about Venus, the planet without a...",4,2701,451,5.988914,24,18.791667,251
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3,2208,373,5.919571,16,23.312500,162


In [10]:
# prompt: Using dataframe essay_df: add a column with sentiment

from textblob import TextBlob
essay_df['sentiment'] = essay_df['full_text'].apply(lambda x: TextBlob(x).sentiment.polarity)


In [15]:
essay_df.head()

,essay_id,full_text,score,text_length,word_count,avg_word_length,sentence_count,average_sentence_length,unique_words,sentiment
0,000d118,Many people have car where they live. The thin...,3,2677,498,5.375502,14,35.571429,253,0.179020
1,000fe60,I am a scientist at NASA that is discussing th...,3,1669,332,5.027108,20,16.600000,177,0.101786
2,001ab80,People always wish they had the same technolog...,4,3077,550,5.594545,25,22.000000,253,0.168435
3,001bdc0,"We all heard about Venus, the planet without a...",4,2701,451,5.988914,24,18.791667,251,0.094291
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3,2208,373,5.919571,16,23.312500,162,0.170952


In [17]:
# prompt: train a model to predict score of the essay

from sklearn.linear_model import LinearRegression

X = essay_df[['sentence_count', 'word_count', 'average_sentence_length', 'unique_words', 'sentiment']]
y = essay_df['score']

model = LinearRegression().fit(X, y)


In [20]:
# prompt: test the model against the test.csv

import pandas as pd
import spacy

nlp = spacy.load('en_core_web_sm')

test_df = pd.read_csv('test.csv')

# Calculate the features for the test set
test_df["sentence_count"] = test_df["full_text"].apply(lambda text: len(list(nlp(text).sents)))
test_df["word_count"] = test_df["full_text"].apply(lambda text: len(text.split()))
test_df['average_sentence_length'] = test_df['word_count'] / test_df['sentence_count']
test_df['unique_words'] = test_df['full_text'].apply(lambda x: len(set(x.split())))
test_df['sentiment'] = test_df['full_text'].apply(lambda x: TextBlob(x).sentiment.polarity)


X_test = test_df[['sentence_count', 'word_count', 'average_sentence_length', 'unique_words', 'sentiment']]
y_pred = model.predict(X_test)

print(y_pred)


[3.37811631 2.81176511 3.80390057]


In [21]:
test_df['score'] = y_pred
test_df.head()

,essay_id,full_text,sentence_count,word_count,average_sentence_length,unique_words,sentiment,score
0,000d118,Many people have car where they live. The thin...,13,498,38.307692,253,0.179020,3.378116
1,000fe60,I am a scientist at NASA that is discussing th...,20,332,16.600000,177,0.101786,2.811765
2,001ab80,People always wish they had the same technolog...,25,550,22.000000,253,0.168435,3.803901
